In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import pathlib
import pretty_midi, mido

C:\Users\lore1\AppData\Local\Temp\ipykernel_19528\333752223.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
cvs_path = 'H:/.shortcut-targets-by-id/1VjZVJYaetZxsv2VWM91CyhZqGyOeiDHX/Music Generator/clean/vgmidi_labelled.csv'

df = pd.read_csv(cvs_path)
df.head()


,id,series,console,game,piece,midi,valence,arousal
0,8013,Banjo-Kazooie,N64,Banjo-Kazooie,Boggys Igloo Happy,labelled/phrases/Banjo-Kazooie_N64_Banjo-Kazoo...,1,1
1,8073,Banjo-Kazooie,N64,Banjo-Kazooie,Boggys Igloo Sad,labelled/phrases/Banjo-Kazooie_N64_Banjo-Kazoo...,1,-1
2,8029,Banjo-Kazooie,N64,Banjo-Kazooie,Bubblegloop Swamp,labelled/phrases/Banjo-Kazooie_N64_Banjo-Kazoo...,1,1
3,8022,Banjo-Kazooie,N64,Banjo-Kazooie,Click Clock Wood,labelled/phrases/Banjo-Kazooie_N64_Banjo-Kazoo...,1,1
4,8066,Banjo-Kazooie,N64,Banjo-Kazooie,Ending,labelled/phrases/Banjo-Kazooie_N64_Banjo-Kazoo...,1,-1


In [4]:
trainFiles = glob.glob('H:/.shortcut-targets-by-id/1VjZVJYaetZxsv2VWM91CyhZqGyOeiDHX/Music Generator/clean/labelled/train/*.mid')
print(len(trainFiles))
testFiles = glob.glob('H:/.shortcut-targets-by-id/1VjZVJYaetZxsv2VWM91CyhZqGyOeiDHX/Music Generator/clean/labelled/test/*.mid')
print(len(testFiles))

110
26


In [8]:
def file_name_format(path, suffix):

    # Split the string using underscores
    parts = path.split('_')

    # Extract relevant parts and create the new string
    new_path = ''
    for part in parts[2:]:
        if '.mid' not in part:
            new_path += part + '_'
        else:
            new_path += part[:-4] + '_' + str(suffix) + part[-4:]
    new_path = 'labelled/phrases/' + new_path
    
    # new_path2 = f'labelled/phrases/{parts[2]}_{parts[3]}_{parts[4]}_{parts[5][:-4]}_{suffix}{parts[5][-4:]}'

    return new_path

def analyze_midi(midi_file):
    
    pm = pretty_midi.PrettyMIDI(midi_file)

    # get tempo changes
    tempo_changes = pm.get_tempo_changes()
    bpm = np.mean(tempo_changes)
    
    # get notes
    notes = []
    for instrument in pm.instruments:
        for note in instrument.notes:
            notes.append(note)
    
    # get notes steps
    sorted_notes = sorted(notes, key=lambda note: note.start)
    prev_start = sorted_notes[0].start
    steps = []
    for note in sorted_notes:
        start = note.start
        steps.append(start - prev_start)
        prev_start = start
    avg_step = np.mean(steps)
    max_step = np.max(steps)


    # get note durations
    durations = [note.end - note.start for note in notes]
    # get average duration
    avg_duration = np.mean(durations)
    max_duration = np.max(durations)

    # get note pitches
    pitches = [note.pitch for note in notes]
    # get average pitch
    avg_pitch = np.mean(pitches)
    max_pitch = np.max(pitches)

    for i in range(3):
        file_row = df.loc[df['midi'] == file_name_format(pathlib.Path(midi_file).name, i)]
        if file_row.shape[0] > 0:
            arousal = file_row['arousal'].values[0]
            valence = file_row['valence'].values[0]
            break

    return {
        'bpm': bpm,
        'avg_step': avg_step,
        'max_step': max_step,
        'avg_duration': avg_duration,
        'max_duration': max_duration,
        'avg_pitch': avg_pitch,
        'max_pitch': max_pitch,
        'valence': valence if valence is not None else 0,
        'arousal': arousal if arousal is not None else 0
    }

resut = analyze_midi(trainFiles[6])
print(resut)

# trainDataset = np.empty((len(trainFiles), 9), dtype=float)
# for file_ind in range(len(trainFiles)):
#     data = analyze_midi(trainFiles[file_ind])

#      # Convert the dictionary values to a NumPy array
#     data_array = np.array(list(data.values())).reshape(1, -1)

#     # Stack the data_array vertically to trainDataset
#     trainDataset[file_ind] = data_array

# print(trainDataset[:5])

{'bpm': 60.0, 'avg_step': 0.1299407114624506, 'max_step': 0.5, 'avg_duration': 0.4683794466403162, 'max_duration': 2.0, 'avg_pitch': 68.75494071146245, 'max_pitch': 78, 'valence': -1, 'arousal': 1}


In [83]:
trainDataset = np.empty((len(trainFiles), 9), dtype=float)
for file_ind in range(len(trainFiles)):
    data = analyze_midi(trainFiles[file_ind])

     # Convert the dictionary values to a NumPy array
    data_array = np.array(list(data.values())).reshape(1, -1)

    # Stack the data_array vertically to trainDataset
    trainDataset[file_ind] = data_array

print('trainDataset shape: ', trainDataset.shape)

testDataset = np.empty((len(testFiles), 9), dtype=float)
for file_ind in range(len(testFiles)):
    data = analyze_midi(testFiles[file_ind])

     # Convert the dictionary values to a NumPy array
    data_array = np.array(list(data.values())).reshape(1, -1)

    # Stack the data_array vertically to trainDataset
    testDataset[file_ind] = data_array

print('testDataset shape: ', testDataset.shape)

trainDataset shape:  (110, 9)
testDataset shape:  (26, 9)


In [84]:
from sklearn.preprocessing import StandardScaler
from  sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = trainDataset[:, :-2]
y_valence = trainDataset[:, -2]
y_arousal = trainDataset[:, -1]

# print('X shape: ', X.shape)
# print('y shape: ', y_valence.shape)
# print('y shape: ', y_arousal.shape)
# print('X: ', X[:5])
# print('y: ', y_valence[:5])
# print('y: ', y_arousal[:5])

# Split the data into training and validation sets
X_train, X_val, y_valence_train, y_valence_val, y_arousal_train, y_arousal_val = train_test_split(
    X, y_valence, y_arousal, test_size=0.1, random_state=42
)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_valence_train.shape)
print("y_val shape:", y_arousal_train.shape)

X_train shape: (99, 7)
X_val shape: (11, 7)
y_train shape: (99,)
y_val shape: (99,)


In [85]:
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier

# Create a classifier: a support vector classifier
svm_valence_classifier = SVC(kernel='linear', C =1.0, tol=1e-3)
svm_arousal_classifier = SVC(kernel='linear', C =1.0, tol=1e-3)

# Fit the classifier to the training data
svm_valence_classifier.fit(X_train, y_valence_train)
svm_arousal_classifier.fit(X_train, y_arousal_train)


# Predict the labels of the validation set
y_valence_pred = svm_valence_classifier.predict(X_val)
y_arousal_pred = svm_arousal_classifier.predict(X_val)

# Compute metrics
print("Accuracy valence: ", accuracy_score(y_valence_val, y_valence_pred))
print("Accuracy arousal: ", accuracy_score(y_arousal_val, y_arousal_pred))

Accuracy valence:  0.45454545454545453
Accuracy arousal:  0.8181818181818182


In [86]:
X_test = testDataset[:, :-2]
y_valence_test = testDataset[:, -2]
y_arousal_test = testDataset[:, -1]

# Predict the labels of the test set
y_valence_pred = svm_valence_classifier.predict(X_test)
y_arousal_pred = svm_arousal_classifier.predict(X_test)

# Compute metrics
print("Accuracy valence: ", accuracy_score(y_valence_test, y_valence_pred))
print("Accuracy arousal: ", accuracy_score(y_arousal_test, y_arousal_pred))

Accuracy valence:  0.5384615384615384
Accuracy arousal:  0.6923076923076923


In [93]:
ind = np.random.randint(0, X_test.shape[0])
sample_data = X_test[ind]
sample_valence = y_valence_test[ind]
sample_arousal = y_arousal_test[ind]
sample_file = testFiles[ind]
sample_valence_pred = svm_valence_classifier.predict(sample_data.reshape(1, -1))
sample_arousal_pred = svm_arousal_classifier.predict(sample_data.reshape(1, -1))

print('True values: ', sample_valence, sample_arousal)
print('Predicted values: ', sample_valence_pred, sample_arousal_pred)

True values:  1.0 1.0
Predicted values:  [1.] [-1.]


## try augmenting the dataset

do small random variations in each midi file keeping the labels to have new files to analyze

In [114]:
midi_file = pretty_midi.PrettyMIDI(trainFiles[0])
print(midi_file.get_pitch_class_histogram())

[0.06823028 0.03411514 0.19189765 0.10234542 0.17697228 0.06183369
 0.0554371  0.09808102 0.02558635 0.09808102 0.04051173 0.04690832]
